In [1]:
# Install necessary for modeling
!pip install transformers --quiet
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import math

# Load the fine-tuned tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(r'C:\Users\sassy\Downloads\Final Project\gpt2-qa')
model = GPT2LMHeadModel.from_pretrained(r'C:\Users\sassy\Downloads\Final Project\gpt2-qa')

# Ensure model is in evaluation mode
model.eval()

# Define the response generation function with response length limitation
def generate_response(conversation_history, max_length=500):
    # Join the conversation history into a single string
    prompt = '\n'.join(conversation_history) + '\nBot:'
    
    # Tokenize prompt
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    
    # Generate a response with a limited max length
    output = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=inputs['input_ids'].shape[1] + 75,
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_p=0.9,
        temperature=0.8
    )
    
    # Decode and extract the bot's response
    response_text = tokenizer.decode(output[0], skip_special_tokens=True)
    generated_text = response_text[len(prompt):].strip()
    for stop_token in ['\nUser:', '\nBot:']:
        generated_text = generated_text.split(stop_token)[0]
    return generated_text.strip()

In [2]:
# Function to truncate conversation history if too long
def truncate_conversation(conversation_history, max_length=1024):
    prompt = '\n'.join(conversation_history) + '\nBot:'
    tokenized_prompt = tokenizer(prompt, return_tensors='pt')
    total_length = tokenized_prompt.input_ids.shape[1]
    while total_length > max_length and len(conversation_history) > 1:
        conversation_history = conversation_history[2:]  # Remove oldest user and bot turn
        tokenized_prompt = tokenizer('\n'.join(conversation_history) + '\nBot:', return_tensors='pt')
        total_length = tokenized_prompt.input_ids.shape[1]
    return conversation_history

# Function to compute log-transformed perplexity to avoid large jumps
def compute_perplexity(model, tokenizer, text):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt')
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        perplexity = torch.exp(loss).item()
    return math.log(perplexity + 1)

# Chat history
conversation_history = []

In [3]:
#Chatbot
print("Type a question. Type 'exit' to quit.")
while True:
    user_input = input("User: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot session ended.")
        break
    
    # User input
    conversation_history.append(f"User: {user_input}")
    
    # Truncate conversation history if necessary
    conversation_history = truncate_conversation(conversation_history, max_length=1024)
    
    # Generate bot response
    bot_response = generate_response(conversation_history)
    
    # Add bot response to conversation history
    conversation_history.append(f"Bot: {bot_response}")
    
    # Print bot response
    print(f"Bot: {bot_response}")
    
    # Compute Perplexity
    perplexity = compute_perplexity(model, tokenizer, bot_response)
    
    # Print Resulting Perplexity
    print(f"\nPerplexity Score (log scale): {perplexity:.2f}")
    print("-" * 50)

Type a question. Type 'exit' to quit.


User:  To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?


Bot: Virgin Mary in L'Anse aux Mêmes

Perplexity Score (log scale): 8.10
--------------------------------------------------


User:  What is in front of the Notre Dame Main Building? Answer: a copper statue of Christ


Bot: A copper statue

Perplexity Score (log scale): 36.94
--------------------------------------------------


...

Bot: Philosophy

Perplexity Score (log scale): 22.94
--------------------------------------------------


User:  What is the philosophy major acceptance rate there?


Bot: 44%

Perplexity Score (log scale): 36.42
--------------------------------------------------


User:  exit


Chatbot session ended.
